In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_store_sku_winare_drq_golive
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS

WITH sku AS (
    SELECT DISTINCT product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_product
    WHERE manufacturer_id IN ('1012', '1461') 
      AND brand_id <> '16009'
),

store AS (
    SELECT DISTINCT store_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
    WHERE UPPER(concept) = 'WIN'
      AND store_status_desc <> 'Đóng cửa'
),

revenue AS (
    SELECT
        a.calday,
        a.store_id,
        SUM(IFNULL(a.revenue.rev_amt_pos, 0)) AS rev_amt_pos
    FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily a
    INNER JOIN store b USING(store_id)
    WHERE a.calday >= DATE_ADD(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), -360)
    GROUP BY 1, 2
    HAVING rev_amt_pos >= 5000000
),

store_golive AS (
    SELECT DISTINCT store_id
    FROM revenue
),

store_sku_list AS (
    SELECT
        store_id, product_id
    FROM store_golive
    CROSS JOIN sku
)

SELECT store_id, product_id
FROM store_sku_list
""")
